In [2]:
import rasterio as reo
from rasterio.enums import Resampling
# from osgeo import gdal


/ihme/homes/nhashmeh/miniconda3/envs/earth-analytics-python/lib/python3.8/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


- LCZ: Local Climate Zones that basically represent a categorical spectrum of urban development (and lack thereof). This dataset is originally at 100m. Source: https://lcz-generator.rub.de/global-lcz-map

In [ ]:
# Paths
lcz_location = '/mnt/share/erf/ERA5/merged_era5_with_stations/LCZ_data/'
lcz_raster = lcz_location + 'lcz_filter_v2.tif'

# resampling LCZ to 1km

In [3]:
# load in data to get shape
og_data = reo.open(lcz_raster)

# factor for rescaling (100m -> 1000m)
downscale_factor = 1/10

with reo.open(lcz_raster, crs='EPSG:4326') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * downscale_factor), # multiply height by downscale factor
            int(dataset.width * downscale_factor) # multiply widrh by downscale factor
        ),
        resampling=Resampling.mode # resampling method
    )

    # scale image transform
    new = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

print('Shape before resample:', og_data.shape)
print('Shape after resample:', data.shape[1:])
print('Transform before resample:\n', dataset.transform, '\n')
print('Transform after resample:\n', new)

new_dataset = reo.open(
    lcz_location + 'lcz_resampled_mode.tif',
    'w',
    driver='GTiff',
    height=data.shape[1],
    width=data.shape[2],
    count=1,
    dtype=data.dtype,
    crs=dataset.crs,
    # crs='EPSG:4326',
    # crs=CRS.from_user_input('EPSG:4326'),
    transform=new,
)

print(new_dataset.shape)

new_dataset.write(data)
new_dataset.close()

Shape before resample: (155995, 389620)
Shape after resample: (15599, 38962)
Transform before resample:
 | 0.00, 0.00,-170.00|
| 0.00,-0.00, 80.04|
| 0.00, 0.00, 1.00| 

Transform after resample:
 | 0.01, 0.00,-170.00|
| 0.00,-0.01, 80.04|
| 0.00, 0.00, 1.00|
(15599, 38962)


ERROR 1: PROJ: proj_create_from_name: /ihme/homes/nhashmeh/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
